## Preprocessing

In [1]:
DATASETS = ["ibtracs.last3years"]

In [2]:
from src.preprocessing.load_data import load_dataset
from src.preprocessing.pipeline import preprocess
from src.preprocessing.split import split_data
from src.preprocessing.save import save_processed, save_data_summary, save_splits
from src.preprocessing.encode import encode

In [3]:
import pandas as pd

In [4]:
for ds in DATASETS:
    df_raw, cfg = load_dataset(ds)
    X_raw, y_raw = preprocess(df_raw.copy(), cfg)
    
    if cfg.get("target_transform") == "binary_landfall":
        y_raw = pd.to_numeric(y_raw, errors="coerce")
        y = (y_raw == 0).astype(int)
        y.name = f"{cfg['target']}_bin"
    else:
        y = y_raw

    X, y = X_raw, y
    save_processed(X, y, ds)
    save_data_summary(X, y, ds)
    splits = split_data(X, y)
    
    splits["X_train"] = encode(splits["X_train"], cfg["preprocessing"].get("encoding", "onehot"))
    splits["X_test"]  = encode(splits["X_test"],  cfg["preprocessing"].get("encoding", "onehot"))

    save_splits(splits, ds)

In [5]:
df_raw.shape, X.shape

((23276, 174), (23276, 31))

In [6]:
y.value_counts(dropna=False)

LANDFALL_bin
0    20975
1     2301
Name: count, dtype: int64